# **POSTagging : Metode Klasifikasi Konvensional Non-Sekuensial**

---
Clarisa Hasya Y - 1301174256


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Read File Data Train TSV**

In [ ]:
df = pd.read_csv("train.01.tsv",sep="\t", header=None)
df = df.astype(str)

In [ ]:
df.head()

,0,1
0,Pemerintah,NN
1,bahkan,RB
2,telah,MD
3,mencanangkan,VB
4,dana,NN


In [ ]:
tmp, temp = [] , [] 
training_sentences, training_tags = [] , []
count_sentences = 0

for index, row in df.iterrows():   
    word = row[0].lower()
    tag = row[1]
    key = (word,tag) # simpan di dictionary kata,tag
    if count_sentences < 50:
        if word != 'nan':
            tmp.append(word)
            temp.append(tag)
            if word == '.' :
                training_sentences.append(tmp)
                training_tags.append(temp)
                tmp = []
                temp = []

    if word == '.' :
        count_sentences += 1      
        if(count_sentences == 50):
            break

In [ ]:
print(training_sentences)

[['pemerintah', 'bahkan', 'telah', 'mencanangkan', 'dana', 'untuk', 'memicu', 'sektor', 'usaha', 'kecil', 'dan', 'menengah', 'ukm', 'tumbuh', 'lebih', 'baik', ',', 'karena', 'sektor', 'ini', 'cukup', 'kuat', 'dalam', 'krisis', 'keuangan', 'pada', 'tahun', '1997', 'lalu', ',', 'kata', '-nya', '.'], ['gubernur', 'bank', 'indonesia', 'burhanudin', 'abdullah', 'optimistis', 'mencapai', 'target', 'proyeksi', 'suku', 'bunga', 'indonesia', 'sbi', '2008', 'yang', 'mencapai', '7,5', '-', '8,0', 'persen', '.'], ['badan', 'kredit', 'desa', 'boleh', 'dibilang', 'sebagai', 'tonggak sejarah', 'berdiri', '-nya', 'lembaga', 'keuangan', 'mikro', 'di', 'indonesia', '.'], ['pada', 'tahun', '1991', 'jumlah', 'utang', 'luar negeri', 'pemerintah', 'membengkak', 'dua', 'kali', 'lipat', 'menjadi', 'us', '$', '45,725', 'miliar', '.'], ['kami', 'sampaikan', 'bahwa', 'pemerintah', 'optimis', 'mengenai', 'tercapai', '-nya', 'target', 'pertumbuhan ekonomi', 'sebesar', '6,2', 'persen', 'dalam', 'tahun', '2009', ','

In [ ]:
print(training_tags)

[['NN', 'RB', 'MD', 'VB', 'NN', 'SC', 'VB', 'NN', 'NN', 'JJ', 'CC', 'JJ', 'NN', 'VB', 'RB', 'JJ', 'Z', 'SC', 'NN', 'PR', 'RB', 'JJ', 'IN', 'JJ', 'NN', 'IN', 'NN', 'CD', 'CC', 'Z', 'VB', 'PRP', 'Z'], ['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'VB', 'NN', 'NN', 'NN', 'NN', 'NNP', 'NNP', 'CD', 'SC', 'VB', 'CD', 'Z', 'CD', 'CD', 'Z'], ['NN', 'NN', 'NN', 'MD', 'VB', 'IN', 'NN', 'VB', 'PRP', 'NN', 'NN', 'JJ', 'IN', 'NNP', 'Z'], ['IN', 'NN', 'CD', 'NN', 'NN', 'NN', 'NN', 'VB', 'CD', 'NND', 'VB', 'VB', 'NNP', 'SYM', 'CD', 'CD', 'Z'], ['PRP', 'VB', 'SC', 'NN', 'NN', 'IN', 'VB', 'PRP', 'NN', 'NN', 'JJ', 'CD', 'CD', 'IN', 'NN', 'CD', 'Z', 'VB', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NN', 'NN', 'PR', 'VB', 'IN', 'NN', 'NNP', 'Z', 'SC', 'NN', 'VB', 'NN', 'NN', 'NN', 'NN', 'PR', 'Z', 'VB', 'PRP', 'Z'], ['SC', 'NN', 'NN', 'VB', 'SYM', 'CD', 'CD', 'SC', 'NN', 'NN', 'JJ', 'SYM', 'CD', 'CD', 'Z'], ['NN', 'NNP', 'MD', 'VB', 'SC', 'NN', 'NN', 'NN', 'MD', 'VB', 'Z', 'CD', 'SC', 'VB', 'NN', 'NN', 'SC', '

**Fungsi untuk Ekstraksi Fitur**


In [ ]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    #print("sentence index = ")
    #print(sentence[index])
    prefix_1 = ''
    prefix_2 = ''
    suffix_1 = ''
    suffix_2 = ''
    if (len(sentence[index])>2):
      prefix_1 = sentence[index][0]
      prefix_2 = sentence[index][:2]
      suffix_1 = sentence[index][-1]
      suffix_2 = sentence[index][-2:]
    return {
        'word': sentence[index],
        'prefix-1': prefix_1,
        'prefix-2': prefix_2,        
        'suffix-1': suffix_1,
        'suffix-2': suffix_2,        
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }

**Fungsi untuk Transformasi Format Dataset**

In [ ]:
def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y

**Read File Data Test TSV**

In [ ]:
dt = pd.read_csv("test_sentences.tsv",sep="\t", header=None)
dt = dt.astype(str)

In [ ]:
dt.head()

,0,1
0,Menteri,NNP
1,pertahanan,NNP
2,AS,NNP
3,dijadwalkan,VB
4,mengunjungi,VB


In [ ]:
tmpp, tempp = [] , [] 
testing_sentences, testing_tags = [] , []

for index, row in dt.iterrows():   
    wordd = row[0].lower()
    tagg = row[1]
    if wordd != 'nan':
        tmpp.append(wordd)
        tempp.append(tagg)
        if wordd == '.' :
            testing_sentences.append(tmpp)
            testing_tags.append(tempp)
            tmpp = []
            tempp = []

In [ ]:
print(testing_sentences)

[['menteri', 'pertahanan', 'as', 'dijadwalkan', 'mengunjungi', 'india', '.'], ['tata', 'power', 'menyuplai', 'batu bara', 'pada', 'tahun', '2000', '.'], ['pemerintah', 'hati-hati', 'dalam', 'mengelola', 'bumn', '.'], ['perusahaan', 'baru', 'tersebut', 'mencanangkan', 'target', 'perolehan', 'laba bersih', '.'], ['menteri', 'pertahanan', 'mengunjungi', 'pangkalan', 'udara', '.'], ['menurut', 'laporan', 'sekretaris', 'perusahaan', ',', 'laba bersih', 'meningkat', '.'], ['transaksi', 'penjualan', 'barang mewah', 'tahun', '2007', 'turun', '.'], ['menkeu', 'memperkirakan', 'inflasi', 'akan', 'meningkat', 'dibanding', 'tahun', 'lalu', '.'], ['kenaikan', 'tarif', 'didorong', 'oleh', 'target', 'laba bersih', 'yang', 'meningkat', '.'], ['makanan', 'dari', 'luar negeri', 'tidak', 'bisa', 'masuk', 'pasar', 'lokal', '.']]


In [ ]:
print(testing_tags)

[['NNP', 'NNP', 'NNP', 'VB', 'VB', 'NNP', 'Z'], ['NNP', 'NNP', 'VB', 'NN', 'IN', 'NN', 'CD', 'Z'], ['NN', 'JJ', 'IN', 'VB', 'NN', 'Z'], ['NN', 'JJ', 'PR', 'VB', 'NN', 'NN', 'NN', 'Z'], ['NNP', 'NNP', 'VB', 'NN', 'NN', 'Z'], ['IN', 'NN', 'NN', 'NN', 'Z', 'NN', 'VB', 'Z'], ['NN', 'NN', 'NN', 'NN', 'CD', 'VB', 'Z'], ['NN', 'VB', 'NN', 'MD', 'VB', 'VB', 'NN', 'CC', 'Z'], ['NN', 'NN', 'VB', 'IN', 'NN', 'NN', 'SC', 'VB', 'Z'], ['NN', 'IN', 'NN', 'NEG', 'MD', 'VB', 'NN', 'JJ', 'Z']]


**Training**

In [ ]:
X, y = transform_to_dataset(training_sentences, training_tags)

from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y)

Pipeline(memory=None,
         steps=[('vectorizer',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('classifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='entropy', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='best'))],
         verbose=False)

**Testing**

In [ ]:
X_test, y_test = transform_to_dataset(testing_sentences, testing_tags)

print('Accuracy Classification: ', clf.score(X_test, y_test))

Accuracy Classification:  0.8571428571428571


**Hasil tagging setiap kalimat uji**

In [ ]:
for sentence in testing_sentences:
    # print(sentence)
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    for i in range(len(sentence)):
        print('kata:',sentence[i] ,', tag:', tags[i])

kata: menteri , tag: RB
kata: pertahanan , tag: NN
kata: as , tag: NNP
kata: dijadwalkan , tag: VB
kata: mengunjungi , tag: VB
kata: india , tag: NNP
kata: . , tag: Z
kata: tata , tag: CD
kata: power , tag: NNP
kata: menyuplai , tag: VB
kata: batu bara , tag: NN
kata: pada , tag: IN
kata: tahun , tag: NN
kata: 2000 , tag: CD
kata: . , tag: Z
kata: pemerintah , tag: NN
kata: hati-hati , tag: JJ
kata: dalam , tag: IN
kata: mengelola , tag: NNP
kata: bumn , tag: NNP
kata: . , tag: Z
kata: perusahaan , tag: NN
kata: baru , tag: JJ
kata: tersebut , tag: PR
kata: mencanangkan , tag: VB
kata: target , tag: NN
kata: perolehan , tag: NN
kata: laba bersih , tag: NN
kata: . , tag: Z
kata: menteri , tag: RB
kata: pertahanan , tag: NN
kata: mengunjungi , tag: VB
kata: pangkalan , tag: NN
kata: udara , tag: NN
kata: . , tag: Z
kata: menurut , tag: IN
kata: laporan , tag: NN
kata: sekretaris , tag: NNP
kata: perusahaan , tag: NN
kata: , , tag: Z
kata: laba bersih , tag: NN
kata: meningkat , tag: VB
k